In [3]:
import onnx
from onnx.helper import *

In [5]:
import onnxruntime as ort

In [4]:
X = make_tensor_value_info('data', TensorProto.FLOAT, [1, 3, 16, 16])

In [6]:
Y = make_tensor_value_info('output', TensorProto.FLOAT, [1, 3,8,8])

In [12]:
node_def = make_node(
    'Resize',                  # name
    ['data',], # inputs
    ['Y'],                  # outputs
    mode='linear'        # attributes
)

In [13]:
graph_def = make_graph(
    [node_def],        # nodes
    'test-resize',      # name
    [X],  # inputs
    [Y],               # outputs
)

In [14]:
model_def = make_model(graph_def, producer_name='onnx-example')

In [15]:
onnx.checker.check_model(model_def)


In [18]:
inferred_model = onnx.shape_inference.infer_shapes(model_def)

RuntimeError: Input 2 is out of bounds.

In [20]:
from torch import nn as nn

In [26]:
 m = nn.Upsample(scale_factor=(0.5,0.5), mode='bilinear')

In [27]:
data = torch.randn(1, 3, 16, 16)

In [28]:
output = m(data)

/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/torch/nn/functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(


In [29]:
output.shape


torch.Size([1, 3, 8, 8])

In [40]:
def CreateDataList(data, count, begin ,truncateBase):
    for i in range(count):
        res = (i + begin) if ((i + begin) < 0) else ((i + begin) % truncateBase)
        data.append(res)

In [30]:
torch.onnx.export(m,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/docs/ByteDance/PNN/converter/quality/upsample.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/torch/nn/functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(
/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/torch/onnx/symbolic_helper.py:381: UserWarning: You are trying to export the model with onnx:Resize for ONNX opset version 10. This operator might cause results to not mat

In [31]:
model_path = "/Users/bob/docs/ByteDance/PNN/converter/quality/upsample.onnx"

In [32]:
import onnxruntime as ort

In [33]:
sess = ort.InferenceSession(model_path)

In [42]:
begin = 1
truncateBase = 13
upsample_data = []
CreateDataList(upsample_data, 1*3*16*16, begin, truncateBase)

In [44]:
upsample_data = np.array(upsample_data,dtype=np.float32).reshape([1,3,16,16])

In [45]:
upsample_data.shape

(1, 3, 16, 16)

In [46]:
outputs  = sess.run(None, {"data" : upsample_data})

In [47]:
outputs[0].flatten()

array([ 1.,  3.,  5.,  7.,  9., 11.,  0.,  2.,  7.,  9., 11.,  0.,  2.,
        4.,  6.,  8.,  0.,  2.,  4.,  6.,  8., 10., 12.,  1.,  6.,  8.,
       10., 12.,  1.,  3.,  5.,  7., 12.,  1.,  3.,  5.,  7.,  9., 11.,
        0.,  5.,  7.,  9., 11.,  0.,  2.,  4.,  6., 11.,  0.,  2.,  4.,
        6.,  8., 10., 12.,  4.,  6.,  8., 10., 12.,  1.,  3.,  5., 10.,
       12.,  1.,  3.,  5.,  7.,  9., 11.,  3.,  5.,  7.,  9., 11.,  0.,
        2.,  4.,  9., 11.,  0.,  2.,  4.,  6.,  8., 10.,  2.,  4.,  6.,
        8., 10., 12.,  1.,  3.,  8., 10., 12.,  1.,  3.,  5.,  7.,  9.,
        1.,  3.,  5.,  7.,  9., 11.,  0.,  2.,  7.,  9., 11.,  0.,  2.,
        4.,  6.,  8.,  0.,  2.,  4.,  6.,  8., 10., 12.,  1.,  6.,  8.,
       10., 12.,  1.,  3.,  5.,  7., 12.,  1.,  3.,  5.,  7.,  9., 11.,
        0.,  5.,  7.,  9., 11.,  0.,  2.,  4.,  6., 11.,  0.,  2.,  4.,
        6.,  8., 10., 12.,  4.,  6.,  8., 10., 12.,  1.,  3.,  5., 10.,
       12.,  1.,  3.,  5.,  7.,  9., 11.,  3.,  5.,  7.,  9., 11

### Winograd

In [2]:
from sympy import Symbol, symbols, Matrix, Poly, zeros, eye, evalf, ccode, Rational

In [7]:
-Rational(1,2)

-1/2

In [6]:
import numpy as np

In [8]:
a = np.array([0,1,2,3,4,5],dtype=np.float32).reshape(3,2)

In [10]:
b = np.array([0,1,2,3,4,5],dtype=np.float32).reshape(2,3)

In [12]:
a@b

array([[ 3.,  4.,  5.],
       [ 9., 14., 19.],
       [15., 24., 33.]], dtype=float32)

In [16]:
bb = np.transpose(b.reshape(3,2),[1,0])

In [17]:
bb

array([[0., 2., 4.],
       [1., 3., 5.]], dtype=float32)

In [19]:
b

array([[0., 1., 2.],
       [3., 4., 5.]], dtype=float32)

In [18]:
a @ bb

array([[ 1.,  3.,  5.],
       [ 3., 13., 23.],
       [ 5., 23., 41.]], dtype=float32)